In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
'''
Useful links:
https://towardsdatascience.com/unsupervised-nlp-topic-models-as-a-supervised-learning-input-cf8ee9e5cf28
https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24
https://markroxor.github.io/gensim/static/notebooks/lda_training_tips.html
https://towardsdatascience.com/end-to-end-topic-modeling-in-python-latent-dirichlet-allocation-lda-35ce4ed6b3e0
https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/
https://datascienceplus.com/evaluation-of-topic-modeling-topic-coherence/

https://towardsdatascience.com/training-and-visualising-word-vectors-2f946c6430f8
https://pdfs.semanticscholar.org/6301/21f9b1f082261e98cede485bfba4f19d59f5.pdf

'''

In [2]:
import numpy as np
import pandas as pd
import gensim
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, RegexpTokenizer
from nltk.stem import WordNetLemmatizer 
nltk.download('wordnet')
import re

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [3]:
train = pd.read_csv('/content/gdrive/My Drive/ClimateTimes/data.csv')


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
train.dropna(subset=['content'])

,id,title,author,date,content,year,month,publication,category,digital,section,url
0,1,Agent Cooper in Twin Peaks is the audience: on...,\nTasha Robinson\n,2017-05-31,And never more so than in Showtime’s new...,2017.0,5.0,Verge,Longform,1.0,NaN,NaN
1,2,"AI, the humanity!",\nSam Byford\n,2017-05-30,AlphaGo’s victory isn’t a defeat for hum...,2017.0,5.0,Verge,Longform,1.0,NaN,NaN
2,3,The Viral Machine,\nKaitlyn Tiffany\n,2017-05-25,Super Deluxe built a weird internet empi...,2017.0,5.0,Verge,Longform,1.0,NaN,NaN
3,4,How Anker is beating Apple and Samsung at thei...,\nNick Statt\n,2017-05-22,Steven Yang quit his job at Google in th...,2017.0,5.0,Verge,Longform,1.0,NaN,NaN
4,5,Tour Black Panther’s reimagined homeland with ...,\nKwame Opam\n,2017-05-15,Ahead of Black Panther’s 2018 theatrical...,2017.0,5.0,Verge,Longform,1.0,NaN,NaN
5,6,Instant recall,\nCasey Newton\n,2017-04-16,Facebook's Instant Articles promised to ...,2017.0,4.0,Verge,Longform,1.0,NaN,NaN
6,7,Massive attack,\nColin Lecher\n,2017-04-14,How a weapon against war became a weapon...,2017.0,4.0,Verge,Longform,1.0,NaN,NaN
7,8,The Future Agency,\nKyle Chayka\n,2017-03-30,Inside the big business of imagining the...,2017.0,3.0,Verge,Longform,1.0,NaN,NaN
8,9,Flight Risk,\nBen Popper\n,2017-03-28,Legal threats and disgruntled clients: i...,2017.0,3.0,Verge,Longform,1.0,NaN,NaN
9,10,Brain drain,\nCasey Newton\n,2017-03-15,Genius quietly laid off a bunch of its e...,2017.0,3.0,Verge,Longform,1.0,NaN,NaN


In [0]:
climate_articles = train.loc[train['content'].str.contains("climate change" , na=False)]

In [6]:
climate_articles.count()

id             4568
title          4568
author         4267
date           4524
content        4568
year           4524
month          4524
publication    4568
category       4255
digital        4513
section        1644
url            2346
dtype: int64

In [7]:
climate_articles = climate_articles['content']
climate_articles.head(10)

7            Inside the big business of imagining the...
16           Past the towers of downtown Miami and ov...
19           The Verge 2021In celebration of our 5th ...
22           On the evening of Saturday, August 27th,...
111      It’s a sunny morning in Camarillo, Californi...
134          &ampldquoJesus,&amprdquo Molly said, her...
194          Over the past 40 years, typhoons that st...
195          Between one Trump tweet after another an...
221          When Donald Trump revealed his energy pl...
283          Marie was feeding her twin daughters whe...
Name: content, dtype: object

In [8]:


climate_articles = [re.sub(r"[^a-zA-Z']",' ',article) for article in climate_articles]
print("Non-alphabet characters removed successfully.")

stop_words = ["a", "about", "above", "after", "again", "against", "ain", "all", "am", "an", "and", "any", "are", "aren", "aren't", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "can", "couldn", "couldn't", "d", "did", "didn", "didn't", "do", "does", "doesn", "doesn't", "doing", "don", "don't", "down", "during", "each", "few", "for", "from", "further", "had", "hadn", "hadn't", "has", "hasn", "hasn't", "have", "haven", "haven't", "having", "he", "her", "here", "hers", "herself", "him", "himself", "his", "how", "i", "if", "in", "into", "is", "isn", "isn't", "it", "it's", "its", "itself", "just", "ll", "m", "ma", "me", "mightn", "mightn't", "more", "most", "mustn", "mustn't", "my", "myself", "needn", "needn't", "no", "nor", "not", "now", "o", "of", "off", "on", "once", "only", "or", "other", "our", "ours", "ourselves", "out", "over", "own", "re", "s", "same", "shan", "shan't", "she", "she's", "should", "should've", "shouldn", "shouldn't", "so", "some", "such", "t", "than", "that", "that'll", "the", "their", "theirs", "them", "themselves", "then", "there", "these", "they", "this", "those", "through", "to", "too", "under", "until", "up", "ve", "very", "was", "wasn", "wasn't", "we", "were", "weren", "weren't", "what", "when", "where", "which", "while", "who", "whom", "why", "will", "with", "won", "won't", "wouldn", "wouldn't", "y", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves", "could", "he'd", "he'll", "he's", "here's", "how's", "i'd", "i'll", "i'm", "i've", "let's", "ought", "she'd", "she'll", "that's", "there's", "they'd", "they'll", "they're", "they've", "we'd", "we'll", "we're", "we've", "what's", "when's", "where's", "who's", "why's", "would", "able", "abst", "accordance", "according", "accordingly", "across", "act", "actually", "added", "adj", "affected", "affecting", "affects", "afterwards", "ah", "almost", "alone", "along", "already", "also", "although", "always", "among", "amongst", "announce", "another", "anybody", "anyhow", "anymore", "anyone", "anything", "anyway", "anyways", "anywhere", "apparently", "approximately", "arent", "arise", "around", "aside", "ask", "asking", "auth", "available", "away", "awfully", "b", "back", "became", "become", "becomes", "becoming", "beforehand", "begin", "beginning", "beginnings", "begins", "behind", "believe", "beside", "besides", "beyond", "biol", "brief", "briefly", "c", "ca", "came", "cannot", "can't", "cause", "causes", "certain", "certainly", "co", "com", "come", "comes", "contain", "containing", "contains", "couldnt", "date", "different", "done", "downwards", "due", "e", "ed", "edu", "effect", "eg", "eight", "eighty", "either", "else", "elsewhere", "end", "ending", "enough", "especially", "et", "etc", "even", "ever", "every", "everybody", "everyone", "everything", "everywhere", "ex", "except", "f", "far", "ff", "fifth", "first", "five", "fix", "followed", "following", "follows", "former", "formerly", "forth", "found", "four", "furthermore", "g", "gave", "get", "gets", "getting", "give", "given", "gives", "giving", "go", "goes", "gone", "got", "gotten", "h", "happens", "hardly", "hed", "hence", "hereafter", "hereby", "herein", "heres", "hereupon", "hes", "hi", "hid", "hither", "home", "howbeit", "however", "hundred", "id", "ie", "im", "immediate", "immediately", "importance", "important", "inc", "indeed", "index", "information", "instead", "invention", "inward", "itd", "it'll", "j", "k", "keep", "keeps", "kept", "kg", "km", "know", "known", "knows", "l", "largely", "last", "lately", "later", "latter", "latterly", "least", "less", "lest", "let", "lets", "like", "liked", "likely", "line", "little", "'ll", "look", "looking", "looks", "ltd", "made", "mainly", "make", "makes", "many", "may", "maybe", "mean", "means", "meantime", "meanwhile", "merely", "mg", "might", "million", "miss", "ml", "moreover", "mostly", "mr", "mrs", "much", "mug", "must", "n", "na", "name", "namely", "nay", "nd", "near", "nearly", "necessarily", "necessary", "need", "needs", "neither", "never", "nevertheless", "new", "next", "nine", "ninety", "nobody", "non", "none", "nonetheless", "noone", "normally", "nos", "noted", "nothing", "nowhere", "obtain", "obtained", "obviously", "often", "oh", "ok", "okay", "old", "omitted", "one", "ones", "onto", "ord", "others", "otherwise", "outside", "overall", "owing", "p", "page", "pages", "part", "particular", "particularly", "past", "per", "perhaps", "placed", "please", "plus", "poorly", "possible", "possibly", "potentially", "pp", "predominantly", "present", "previously", "primarily", "probably", "promptly", "proud", "provides", "put", "q", "que", "quickly", "quite", "qv", "r", "ran", "rather", "rd", "readily", "really", "recent", "recently", "ref", "refs", "regarding", "regardless", "regards", "related", "relatively", "research", "respectively", "resulted", "resulting", "results", "right", "run", "said", "saw", "say", "saying", "says", "sec", "section", "see", "seeing", "seem", "seemed", "seeming", "seems", "seen", "self", "selves", "sent", "seven", "several", "shall", "shed", "shes", "show", "showed", "shown", "showns", "shows", "significant", "significantly", "similar", "similarly", "since", "six", "slightly", "somebody", "somehow", "someone", "somethan", "something", "sometime", "sometimes", "somewhat", "somewhere", "soon", "sorry", "specifically", "specified", "specify", "specifying", "still", "stop", "strongly", "sub", "substantially", "successfully", "sufficiently", "suggest", "sup", "sure", "take", "taken", "taking", "tell", "tends", "th", "thank", "thanks", "thanx", "thats", "that've", "thence", "thereafter", "thereby", "thered", "therefore", "therein", "there'll", "thereof", "therere", "theres", "thereto", "thereupon", "there've", "theyd", "theyre", "think", "thou", "though", "thoughh", "thousand", "throug", "throughout", "thru", "thus", "til", "tip", "together", "took", "toward", "towards", "tried", "tries", "truly", "try", "trying", "ts", "twice", "two", "u", "un", "unfortunately", "unless", "unlike", "unlikely", "unto", "upon", "ups", "us", "use", "used", "useful", "usefully", "usefulness", "uses", "using", "usually", "v", "value", "various", "'ve", "via", "viz", "vol", "vols", "vs", "w", "want", "wants", "wasnt", "way", "wed", "welcome", "went", "werent", "whatever", "what'll", "whats", "whence", "whenever", "whereafter", "whereas", "whereby", "wherein", "wheres", "whereupon", "wherever", "whether", "whim", "whither", "whod", "whoever", "whole", "who'll", "whomever", "whos", "whose", "widely", "willing", "wish", "within", "without", "wont", "words", "world", "wouldnt", "www", "x", "yes", "yet", "youd", "youre", "z", "zero", "a's", "ain't", "allow", "allows", "apart", "appear", "appreciate", "appropriate", "associated", "best", "better", "c'mon", "c's", "cant", "changes", "clearly", "concerning", "consequently", "consider", "considering", "corresponding", "course", "currently", "definitely", "described", "despite", "entirely", "exactly", "example", "going", "greetings", "hello", "help", "hopefully", "ignored", "inasmuch", "indicate", "indicated", "indicates", "inner", "insofar", "it'd", "keep", "keeps", "novel", "presumably", "reasonably", "second", "secondly", "sensible", "serious", "seriously", "sure", "t's", "third", "thorough", "thoroughly", "three", "well", "wonder", "a", "about", "above", "above", "across", "after", "afterwards", "again", "against", "all", "almost", "alone", "along", "already", "also", "although", "always", "am", "among", "amongst", "amoungst", "amount", "an", "and", "another", "any", "anyhow", "anyone", "anything", "anyway", "anywhere", "are", "around", "as", "at", "back", "be", "became", "because", "become", "becomes", "becoming", "been", "before", "beforehand", "behind", "being", "below", "beside", "besides", "between", "beyond", "bill", "both", "bottom", "but", "by", "call", "can", "cannot", "cant", "co", "con", "could", "couldnt", "cry", "de", "describe", "detail", "do", "done", "down", "due", "during", "each", "eg", "eight", "either", "eleven", "else", "elsewhere", "empty", "enough", "etc", "even", "ever", "every", "everyone", "everything", "everywhere", "except", "few", "fifteen", "fify", "fill", "find", "fire", "first", "five", "for", "former", "formerly", "forty", "found", "four", "from", "front", "full", "further", "get", "give", "go", "had", "has", "hasnt", "have", "he", "hence", "her", "here", "hereafter", "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his", "how", "however", "hundred", "ie", "if", "in", "inc", "indeed", "interest", "into", "is", "it", "its", "itself", "keep", "last", "latter", "latterly", "least", "less", "ltd", "made", "many", "may", "me", "meanwhile", "might", "mill", "mine", "more", "moreover", "most", "mostly", "move", "much", "must", "my", "myself", "name", "namely", "neither", "never", "nevertheless", "next", "nine", "no", "nobody", "none", "noone", "nor", "not", "nothing", "now", "nowhere", "of", "off", "often", "on", "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our", "ours", "ourselves", "out", "over", "own", "part", "per", "perhaps", "please", "put", "rather", "re", "same", "see", "seem", "seemed", "seeming", "seems", "serious", "several", "she", "should", "show", "side", "since", "sincere", "six", "sixty", "so", "some", "somehow", "someone", "something", "sometime", "sometimes", "somewhere", "still", "such", "system", "take", "ten", "than", "that", "the", "their", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "therefore", "therein", "thereupon", "these", "they", "thickv", "thin", "third", "this", "those", "though", "three", "through", "throughout", "thru", "thus", "to", "together", "too", "top", "toward", "towards", "twelve", "twenty", "two", "un", "under", "until", "up", "upon", "us", "very", "via", "was", "we", "well", "were", "what", "whatever", "when", "whence", "whenever", "where", "whereafter", "whereas", "whereby", "wherein", "whereupon", "wherever", "whether", "which", "while", "whither", "who", "whoever", "whole", "whom", "whose", "why", "will", "with", "within", "without", "would", "yet", "you", "your", "yours", "yourself", "yourselves", "the", "a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z", "A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z", "co", "op", "research-articl", "pagecount", "cit", "ibid", "les", "le", "au", "que", "est", "pas", "vol", "el", "los", "pp", "well-b", "http", "volumtype", "par", "ab", "ac", "ad", "ae", "af", "ag", "aj", "al", "an", "ao", "ap", "ar", "av", "aw", "ax", "ay", "az", "b1", "b2", "b3", "ba", "bc", "bd", "be", "bi", "bj", "bk", "bl", "bn", "bp", "br", "bs", "bt", "bu", "bx", "c1", "c2", "c3", "cc", "cd", "ce", "cf", "cg", "ch", "ci", "cj", "cl", "cm", "cn", "cp", "cq", "cr", "cs", "ct", "cu", "cv", "cx", "cy", "cz", "d2", "da", "dc", "dd", "de", "df", "di", "dj", "dk", "dl", "do", "dp", "dr", "ds", "dt", "du", "dx", "dy", "e2", "e3", "ea", "ec", "ed", "ee", "ef", "ei", "ej", "el", "em", "en", "eo", "ep", "eq", "er", "es", "et", "eu", "ev", "ex", "ey", "f2", "fa", "fc", "ff", "fi", "fj", "fl", "fn", "fo", "fr", "fs", "ft", "fu", "fy", "ga", "ge", "gi", "gj", "gl", "go", "gr", "gs", "gy", "h2", "h3", "hh", "hi", "hj", "ho", "hr", "hs", "hu", "hy", "i", "i2", "i3", "i4", "i6", "i7", "i8", "ia", "ib", "ic", "ie", "ig", "ih", "ii", "ij", "il", "in", "io", "ip", "iq", "ir", "iv", "ix", "iy", "iz", "jj", "jr", "js", "jt", "ju", "ke", "kg", "kj", "km", "ko", "l2", "la", "lb", "lc", "lf", "lj", "ln", "lo", "lr", "ls", "lt", "m2", "ml", "mn", "mo", "ms", "mt", "mu", "n2", "nc", "nd", "ne", "ng", "ni", "nj", "nl", "nn", "nr", "ns", "nt", "ny", "oa", "ob", "oc", "od", "of", "og", "oi", "oj", "ol", "om", "on", "oo", "oq", "or", "os", "ot", "ou", "ow", "ox", "oz", "p1", "p2", "p3", "pc", "pd", "pe", "pf", "ph", "pi", "pj", "pk", "pl", "pm", "pn", "po", "pq", "pr", "ps", "pt", "pu", "py", "qj", "qu", "r2", "ra", "rc", "rd", "rf", "rh", "ri", "rj", "rl", "rm", "rn", "ro", "rq", "rr", "rs", "rt", "ru", "rv", "ry", "s2", "sa", "sc", "sd", "se", "sf", "si", "sj", "sl", "sm", "sn", "sp", "sq", "sr", "ss", "st", "sy", "sz", "t1", "t2", "t3", "tb", "tc", "td", "te", "tf", "th", "ti", "tj", "tl", "tm", "tn", "tp", "tq", "tr", "ts", "tt", "tv", "tx", "ue", "ui", "uj", "uk", "um", "un", "uo", "ur", "ut", "va", "wa", "vd", "wi", "vj", "vo", "wo", "vq", "vt", "vu", "x1", "x2", "x3", "xf", "xi", "xj", "xk", "xl", "xn", "xo", "xs", "xt", "xv", "xx", "y2", "yj", "yl", "yr", "ys", "yt", "zi", "zz"]
stop_re = '\\b'+'\\b|\\b'.join(stop_words)+'\\b'
climate_articles = [re.sub(r'{}'.format(stop_re), '',article.lower()) for article in climate_articles]
print("Stop words removed successfully.")


climate_articles[:2]


Non-alphabet characters removed successfully.
Stop words removed successfully.


["      inside  big business  imagining  future   dubai            instantly familiar  utterly strange    urban planning  driven   omniscient ai installed      skyscraper  smart bathroom mirror tracks  physical health   interface   government   personalized  genie    hologram   form   virtual emirati gentleman  traditional garb     future products  skinned    visual aesthetic  friendly white  black animated icons   minimalist  sentient version  apple  ios  uncanny scene   display   united arab emirates   annual government summit  hosted  dubai  february         day event comprised  dozens  speakers   including sir richard branson           participants   bills     largest annual government gathering       gathering  meant   build hope  build life  future    people happy    sheikh mohammed bin rashid  maktoum  prime minister   uae  royal ruler  dubai    event   urban ai  hologram genie   smart bathroom     museum  future government services   series  seamless interactive installations  

In [9]:
articles = [article.split() for article in climate_articles]
print("Words tokenized successfully.")

Words tokenized successfully.


In [11]:
print("Initializing lemmatizer...")
lemmatizer = WordNetLemmatizer() 
lemm_articles=[]
for article in articles:
	  temp = [lemmatizer.lemmatize(word) for word in article]
	  lemm_articles.append(temp)
print("Lemmatization finished successfully.")

#Input articles through Phraser
phrases = gensim.models.phrases.Phrases(lemm_articles)
print("Phrases completed.")
phraser = gensim.models.phrases.Phraser(phrases)
train_phrased = phraser[lemm_articles]
print("Phrasing process completed!")

Initializing lemmatizer...
Lemmatization finished successfully.
Phrases completed.
Phrasing process completed!


## LDA Testing

In [12]:
!pip install pyLDAvis

     |████████████████████████████████| 1.6MB 4.8MB/s 
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyLDAvis


In [0]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline


In [0]:
#Create dictionary
dictionary = gensim.corpora.Dictionary(train_phrased)

In [0]:
#Term-document Frequency
corpus = [dictionary.doc2bow(article) for text in train_phrased]

In [59]:
# Human readable format of corpus (term-frequency)
[[(dictionary[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('asked', 1),
  ('body', 4),
  ('called', 1),
  ('chair', 1),
  ('collapse', 1),
  ('completed', 1),
  ('day', 1),
  ('decade', 1),
  ('deep', 1),
  ('edge', 2),
  ('experience', 1),
  ('experienced', 3),
  ('fever', 1),
  ('good', 1),
  ('health', 1),
  ('high', 1),
  ('hold', 1),
  ('hope', 1),
  ('hour', 1),
  ('human', 1),
  ('interview', 1),
  ('life', 2),
  ('ocean', 4),
  ('people', 4),
  ('problem', 1),
  ('produced', 1),
  ('rising', 1),
  ('set', 1),
  ('sound', 1),
  ('stayed', 1),
  ('temperature', 3),
  ('thing', 1),
  ('time', 2),
  ('united', 1),
  ('water', 5),
  ('year', 2),
  ('africa', 1),
  ('cape', 1),
  ('change', 1),
  ('climate', 1),
  ('close', 1),
  ('coast', 1),
  ('coming', 1),
  ('fresh', 1),
  ('glacier', 1),
  ('heat', 1),
  ('island', 1),
  ('land', 1),
  ('leg', 1),
  ('long', 2),
  ('low', 1),
  ('marine', 1),
  ('north', 1),
  ('northern', 1),
  ('place', 2),
  ('pulled', 1),
  ('rise', 1),
  ('risen', 1),
  ('sea', 1),
  ('south', 1),
  ('spent', 1

In [0]:
#build LDA model
#https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=10,
                                           passes=100,
                                           alpha='auto',
                                           per_word_topics=True)

Output hidden; open in https://colab.research.google.com to view.

In [68]:
print(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0, '0.044*"pugh" + 0.024*"swim" + 0.020*"water" + 0.020*"arctic" + 0.016*"body" + 0.016*"people" + 0.016*"saturday" + 0.016*"cold" + 0.016*"ocean" + 0.012*"ice"'), (1, '0.000*"serving_dettinger" + 0.000*"pretty_wild" + 0.000*"spillway_nearing" + 0.000*"southwest_drier" + 0.000*"south_tilt" + 0.000*"south_dragged" + 0.000*"snowpack_slowly" + 0.000*"simply_bang" + 0.000*"shift_atmospheric" + 0.000*"storm_drift"'), (2, '0.000*"serving_dettinger" + 0.000*"pretty_wild" + 0.000*"spillway_nearing" + 0.000*"southwest_drier" + 0.000*"south_tilt" + 0.000*"south_dragged" + 0.000*"snowpack_slowly" + 0.000*"simply_bang" + 0.000*"shift_atmospheric" + 0.000*"storm_drift"'), (3, '0.000*"serving_dettinger" + 0.000*"pretty_wild" + 0.000*"spillway_nearing" + 0.000*"southwest_drier" + 0.000*"south_tilt" + 0.000*"south_dragged" + 0.000*"snowpack_slowly" + 0.000*"simply_bang" + 0.000*"shift_atmospheric" + 0.000*"storm_drift"'), (4, '0.000*"serving_dettinger" + 0.000*"pretty_wild" + 0.000*"spillway_nearing

In [0]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)
vis